In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from multiprocessing  import Pool

from typing import Sequence
import itertools
import pytest
# ! pip install pyspark

import dask.dataframe as dd

#parallel
import psutil
import time
import os
from pathlib import Path
# ! pip install multiprocess 

### Create all features given a k-mer length

In [2]:
#Define Kmer size 
k=6
features = []

def generate_all_kmers(k: int, alphabet="CGAT") -> Sequence[str]:
    """generate all polymers of length k
    Args:
        k: length of the polymer
        alphabet: base unit of the kmer, [C, G, A, T] for DNA
    """

    def _generate_all_kmers(partial_kmers: Sequence[str]):
        if len(partial_kmers[0]) == k:
            return partial_kmers
        return _generate_all_kmers(
            partial_kmers=list(
                itertools.chain.from_iterable(
                    [partial_kmer + n for n in alphabet]
                    for partial_kmer in partial_kmers
                )
            )
        )

    return _generate_all_kmers(alphabet)

features = generate_all_kmers(k, alphabet="ACGT")

In [3]:
len(features)
# display?

### Import kmer count files and transform into dataframe

In [4]:
import defs

In [ ]:
import time
k=6
df = pd.DataFrame(columns=features)
display(df)
# countsPath = r'D:\DataSet\MULTI\test bow\raw count'
# countsPath = r"D:\DataSet\MULTI\bow\\" + str(k)+ r"\mer\600"
countsPath = r"D:\DataSet\MULTI\bow\6mer\6"


def process_file(filename):
    # i = 0
#for i in range(0, 600):
    print("this is process file, fileaname:", filename)
    sample = pd.read_fwf(filename,sep=" ", header=None).T
    display("sample : ", sample)
    new_header = sample.iloc[0] #grab the first row for the header
    sample = sample[1:] #take the data less the header row
    sample.columns = new_header #set the header row as the df header

    df= df.append(sample, ignore_index=True)  #APPEND Sample to df DataSet
    display(df)
    return df
def get_files(directory,pattern):
    '''
    Get the files of a directory
    '''
    for path in Path(directory).rglob(pattern):
        yield path.absolute()
# def process_file(filename):
#     ''''
#     Read an xls file, retun a dataframe   
#     ''''
#     return pd.read_csv(filename,index_col=None)
def pd_wrapper(directory,pattern,processes=-1):
    # Decide how many proccesses will be created
    sum_size = 0
    if processes <=0:
        num_cpus = psutil.cpu_count(logical=False)
    else:
        num_cpus = processes
        files = []
        # Get files based on pattern and their sum of size
        for file in get_files(directory=directory,pattern=pattern):
                sum_size =sum_size + os.path.getsize(file)
                files.append(file)
#         display(files)
        print('files:%s,size:%s bytes, procs:%s'%(len(files),sum_size,num_cpus))
        # Create the pool
        process_pool = Pool(processes=num_cpus)
        start = time.time()
        # Start processes in the pool
        dfs = process_pool.map(process_file, files)
        # Concat dataframes to one dataframe
        data = pd.concat(dfs, ignore_index=True)
        end = time.time()
        print('Completed in: %s sec'%(end - start))
        print("data.type", type(data))
        return data
if __name__ == '__main__':
    start = time.time()
    df = pd_wrapper(directory=countsPath,pattern='*.txt',processes=8)
    print(df.count)
    print(type(df))

    end = time.time()
    # total time taken
    print(f"Runtime of the program is {end - start} secs")


#     display(sample)
display(df)
# df.head

In [ ]:
df.to_feather(r"D:\DataSet\MULTI\bow\df-k"+str(k)+".feather")

### merge sample into single dataframe  using Dask
## Result: This approach will not work as Dask doesn't support inserting series with missing values (unlike pandas, which fills Nan for them).
### *ValueError: Length mismatch: Expected axis has 1455 elements, new values have 3072 elements

## Transpose: This code transposes files, prerequisite for parallel approach.